In [1]:
import dspy
import random

max_tokens = 2000
temperature = 0.6 + random.random() * 0.4
frequency_penalty = 0.5
model = "llama3"

ollama = dspy.OllamaLocal(
    model=model,
    max_tokens=max_tokens,
    temperature=temperature,
    frequency_penalty=frequency_penalty,
    timeout_s=500,
)

gpt3 = dspy.OpenAI(
    model="gpt-3.5-turbo",
    max_tokens=max_tokens,
    temperature=temperature,
    # system_prompt=system_prompt,
)

dspy.configure(lm=gpt3)

In [2]:
from adventure.utils import print_divider, strip_numbered_prefixes
from adventure.modules import (
    PointOfInterestModule,
    EncounterModule,
    EncounterMenuModule,
    EncounterResolverModule,
    GameStartModule,
)

game_start_generator = GameStartModule()
points_of_interest_generator = PointOfInterestModule()

encounter_generator = EncounterModule()
encounter_menu_generator = EncounterMenuModule()
encounter_outcome_resolver = EncounterResolverModule()

player_character = "Draehus"
game_description = "The land of the Red Dragon."

game_context = game_start_generator(
    player_character=player_character, game_description=game_description
)

print_divider(f"Backstory ({player_character})")
print(game_context.backstory)
print_divider()

print_divider(f"Narrative ({game_description[:20].strip()}...)")
print(game_context.narrative)
print_divider()

print_divider(f"Summary")
print(game_context.summary)
print_divider()

print_divider(f"Cities")
for city in game_context.cities_towns:
    print(city)
print_divider()

print_divider(f"Wilds")
for wild in game_context.wilds:
    print(wild)
print_divider()


- Backstory (Draehus) ----------------------------------------------------------
Born in the fiery depths of the Red Dragon's lair, Draehus has always felt a connection to the ancient beasts that rule this land. Trained in the ways of combat and magic from a young age, he now seeks to prove himself as a worthy warrior and protector of his homeland. With his fierce determination and unwavering loyalty, Draehus will stop at nothing to defend his people and vanquish any who dare to threaten the Red Dragon's domain.
--------------------------------------------------------------------------------

- Narrative (The land of the Red...) -------------------------------------------
Born in the fiery depths of the Red Dragon's lair, Draehus was raised among the flames and shadows of his powerful kin. Trained relentlessly in combat and magic, he honed his skills and grew into a formidable warrior with a deep sense of loyalty to his people. As he ventured out into the world, Draehus faced countles

In [3]:
for location in game_context.cities_towns:
    location = strip_numbered_prefixes(location)

    points_of_interest = points_of_interest_generator(
        narrative=game_context.narrative, primary_location=location
    )

    print_divider(f"Points Of Interest ({location})")
    for poi in points_of_interest:
        print(poi)
    print_divider()


- Points Of Interest (Flamecrest) ----------------------------------------------
The Red Dragon's Lair
The Hall of Flames
The Shadow Ridge
The Dragon's Breath Gorge
The Shrine of Ancient Fire
--------------------------------------------------------------------------------

- Points Of Interest (Shadowvale) ----------------------------------------------
The Red Dragon's Lair
The Flames of the Forge
The Shadows of the Ancient Library
The Fiery Peak of Mount Ember
The Hall of Dragon's Breath
--------------------------------------------------------------------------------

- Points Of Interest (Dragonfire Peak) -----------------------------------------
The Red Dragon's Lair
The Flameshadow Caves
The Fiery Citadel
The Dragon's Roost
The Lava Fields
--------------------------------------------------------------------------------

- Points Of Interest (Emberglow) -----------------------------------------------
The Red Dragon's Lair
The Flameshadow Arena
The Ancient Dragon's Tomb
Draconic Pea

In [4]:
print_divider(f"Random Location ({game_context.current_location})")
print(game_context.current_location)
print_divider()

encounter_text = encounter_generator(
    context=game_context.summary,
    location=game_context.current_location,
    player_character=player_character,
)

print_divider(f"Encounter")
print(encounter_text)
print_divider()


- Random Location (Shadowed Canyon Sanctuary) ----------------------------------
Shadowed Canyon Sanctuary
--------------------------------------------------------------------------------

- Encounter --------------------------------------------------------------------
As Draehus approaches the entrance to the Shadowed Canyon Sanctuary, he senses a disturbance in the air. The faint smell of sulfur mixed with the sound of distant whispers fills the air, causing his dragon blood to boil with anticipation. Suddenly, a group of rogue sorcerers emerge from the shadows, their eyes glowing with dark power as they prepare to unleash their magic upon the sanctuary. Draehus must use his combat skills and powerful magic to protect his homeland and defeat the rogue sorcerers before they can cause any harm.
--------------------------------------------------------------------------------


In [5]:
menu_options = encounter_menu_generator(encounter=encounter_text)

print_divider(f"Menu")
for item in menu_options:
    print(item)
print_divider()


- Menu -------------------------------------------------------------------------
Engage the rogue sorcerers in combat 
Use magic to create a protective barrier around the sanctuary 
Attempt to reason with the rogue sorcerers and convince them to leave peacefully 
Call upon the help of the sanctuary's guardians to aid in the battle
--------------------------------------------------------------------------------


In [6]:
random_option = random.choice(menu_options)
player_choice = f"{player_character} chooses to: {random_option}"

print_divider(f"Random Option")
print(player_choice)
print_divider()

# Roll a D20 to determine succes
d20_roll = random.randint(1, 20)
player_rolls = f"{player_character} rolls a {d20_roll} out of 20 odds of success."

print_divider(f"Player Rolls")
print(player_rolls)
print_divider()

context = f"{encounter_text}\n\n{player_choice}\n\n{player_rolls}"

new_encounter_text = encounter_generator(
    context=context,
    location=game_context.current_location,
    player_character=player_character,
)

print_divider(f"Encounter")
print(new_encounter_text)
print_divider()


- Random Option ----------------------------------------------------------------
Draehus chooses to: Attempt to reason with the rogue sorcerers and convince them to leave peacefully 
--------------------------------------------------------------------------------

- Player Rolls -----------------------------------------------------------------
Draehus rolls a 17 out of 20 odds of success.
--------------------------------------------------------------------------------

- Encounter --------------------------------------------------------------------
As Draehus steps forward, holding his hands out in a gesture of peace, he begins to speak to the rogue sorcerers. His words are filled with conviction and power, urging them to reconsider their actions and leave the sanctuary unharmed. The sorcerers pause, their eyes flickering with uncertainty as they listen to Draehus's words. After a tense moment, one of the sorcerers nods and signals to the others to stand down. The group slowly backs a

In [7]:
encounter_details = f"{player_choice}\n\n{new_encounter_text}"

outcome = encounter_outcome_resolver(
    encounter=encounter_details, player_character=player_character
)

print_divider(f"Outcome")
print(outcome)
print_divider()


- Outcome ----------------------------------------------------------------------
resolved_success
--------------------------------------------------------------------------------
